### Import model and use on testing csv
This is rough but its working

In [1]:
## Phase 0/1 Code.
import scipy.cluster.hierarchy as sch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler

dfTest = pd.read_csv("Phishing_Legitimate_test_student.csv", na_values=['',' ','n/a'])

#### Data cleaning & standardization of Testing dataset

In [2]:
## cleaning testing data ##
testRowsWithNa = dfTest[ dfTest.isnull().any(axis=1) ]
rowsToDrop = dfTest[ dfTest.isnull().sum(axis=1) > 1 ].index
dfTest.drop(rowsToDrop, inplace=True)
imputer = KNNImputer(n_neighbors=10)
dfTest2 = pd.DataFrame(imputer.fit_transform(dfTest),columns = dfTest.columns)
rows_to_drop=dfTest2[dfTest2['UrlLength']>500].index
dfTest2.drop(rows_to_drop,inplace=True)
rows_to_drop=dfTest2[dfTest2['NumNumericChars']>100].index
dfTest2.drop(rows_to_drop,inplace=True)
rows_to_drop=dfTest2[dfTest2['NumDash']>20].index
dfTest2.drop(rows_to_drop,inplace=True)
clf= LocalOutlierFactor(n_neighbors=20)
X=dfTest2.to_numpy()
outlier_label=clf.fit_predict(X)
rows_to_drop= dfTest2.iloc[clf.negative_outlier_factor_ < -1.30].index
dfTest2.drop(rows_to_drop,inplace=True)

Create dataframe of selected features, must be in proper order

In [3]:
X=dfTest2[['NumDash','PathLength','UrlLength','PathLevel','DomainInPaths','RandomString','InsecureForms']].to_numpy()

# Scale/standardize data 
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

#### Import & Build model using pickle

In [4]:
import pickle
load_model=pickle.load(open('test','rb'))

# Test same model on different dataset 
y=load_model.predict(X) # Build prediction model

##### Export predictions to CSV file

In [5]:
### export to csv ###
ids = dfTest2['id'].astype(int)
# DF created to output the csv file in correct format
final_predictions = pd.DataFrame(y,ids).rename_axis('Id')
final_predictions.columns = ['Prediction']
# must change file path 
final_predictions.to_csv('test_export.csv')

## Below was just to see how the imported model did the the entire TRAINING dataset. This can be removed for submittal

In [11]:
######### TEST IMPORTED MODEL ON ENTIRE TRAINING SET ########### 
### Test model on training dataset ### This can be removed prior to submittal
### Phase 0/1 Code.
import scipy.cluster.hierarchy as sch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler


dfTest = pd.read_csv("Phishing_Legitimate_train_missing_data.csv", na_values=['',' ','n/a'])

##### Data Cleaning of Test csv ##

## cleaning & standardizing training data ##
testRowsWithNa = dfTest[ dfTest.isnull().any(axis=1) ]
rowsToDrop = dfTest[ dfTest.isnull().sum(axis=1) > 1 ].index
dfTest.drop(rowsToDrop, inplace=True)
imputer = KNNImputer(n_neighbors=10)
dfTest2 = pd.DataFrame(imputer.fit_transform(dfTest),columns = dfTest.columns)
rows_to_drop=dfTest2[dfTest2['UrlLength']>500].index
dfTest2.drop(rows_to_drop,inplace=True)
rows_to_drop=dfTest2[dfTest2['NumNumericChars']>100].index
dfTest2.drop(rows_to_drop,inplace=True)
rows_to_drop=dfTest2[dfTest2['NumDash']>20].index
dfTest2.drop(rows_to_drop,inplace=True)
clf= LocalOutlierFactor(n_neighbors=20)
X=dfTest2.to_numpy()
outlier_label=clf.fit_predict(X)
rows_to_drop= dfTest2.iloc[clf.negative_outlier_factor_ < -1.30].index
dfTest2.drop(rows_to_drop,inplace=True)

# Dataframe of selected features, must be loaded in same order as in training set
X=dfTest2[['NumDash','PathLength','UrlLength','PathLevel','DomainInPaths','RandomString','InsecureForms']].to_numpy()

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

# Import model
import pickle
load_model=pickle.load(open('test','rb'))

# Test same model on different dataset 

y=load_model.predict(X) # Build prediction model
Y=dfTest2['CLASS_LABEL'].to_numpy()

### export to csv ###
#ids = dfTest2['id'].astype(int)
# DF created to output the csv file in correct format
#final_predictions = pd.DataFrame(y,ids).rename_axis('Id')
#final_predictions.columns = ['Prediction']

# must change file path 
#final_predictions.to_csv('test_export.csv')

# Test model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


print("Accuracy score: ",accuracy_score(Y, y)) # LoL bad
print('Confusion Matrix', confusion_matrix(Y, y)) # Big Problem with FN

Accuracy score:  0.537809187279152
Confusion Matrix [[1977   71]
 [1891  306]]


not that good, problem with FN 